## RNN을 이용한 텍스트 생성

In [1]:
import numpy as np
import tensorflow as tf
seed = 2023
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
text = '''경마장에 있는 말이 뛰고 있다.
그의 말이 법이다.
가는 말이 고와야 오는 말이 곱다.'''

#### 텍스트 전처리

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [4]:
# 단어 집합 생성
t = Tokenizer()
t.fit_on_texts([text])

In [5]:
# 특징
#   1. 사전 순이 아니고, 많이 나온 순서
#   2. 1에서 시작함. (0 은 padding용)
t.word_index

{'말이': 1,
 '경마장에': 2,
 '있는': 3,
 '뛰고': 4,
 '있다': 5,
 '그의': 6,
 '법이다': 7,
 '가는': 8,
 '고와야': 9,
 '오는': 10,
 '곱다': 11}

In [6]:
# 단어집합 크기 - 시작 인덱스가 1부터 시작하므로, 0(패딩)을 처리하기 위해 1을 더해줌
vocab_size = len(t.word_index) + 1
vocab_size

12

In [7]:
t.texts_to_sequences(['그의 말이 법이다.'])

[[6, 1, 7]]

In [8]:
# 주어진 텍스트(corpus)를 encoding한 결과
for line in text.split('\n'):
    encoded = t.texts_to_sequences([line])[0]
    print(encoded)

[2, 3, 1, 4, 5]
[6, 1, 7]
[8, 1, 9, 10, 1, 11]


- sequence 생성

In [9]:
sequences = []
for line in text.split('\n'):
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequences.append(encoded[:i+1])
sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

In [10]:
# sequence 중에서 제일 길이가 긴 것
max_len = max(len(seq) for seq in sequences)
max_len

6

- Padding
    - 전체 샘플 길이를 max_len에 맞추도록 0을 추가
    - 앞쪽을 채우는 pre 옵션이 디폴트

In [11]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
sequences

array([[ 0,  0,  0,  0,  2,  3],
       [ 0,  0,  0,  2,  3,  1],
       [ 0,  0,  2,  3,  1,  4],
       [ 0,  2,  3,  1,  4,  5],
       [ 0,  0,  0,  0,  6,  1],
       [ 0,  0,  0,  6,  1,  7],
       [ 0,  0,  0,  0,  8,  1],
       [ 0,  0,  0,  8,  1,  9],
       [ 0,  0,  8,  1,  9, 10],
       [ 0,  8,  1,  9, 10,  1],
       [ 8,  1,  9, 10,  1, 11]])

- X, y 데이터 설정

In [12]:
X = sequences[:, :-1]
y = sequences[:, -1]
X.shape, y.shape

((11, 5), (11,))

In [13]:
Y = to_categorical(y)
Y.shape

(11, 12)

#### 모델 생성/설정/학습/평가
- Embedding
- SimpleRNN
- Dense

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [15]:
# Embedding vector의 length=4, SimpleRNN node수=32
model = Sequential([
    Embedding(vocab_size, 4, input_length=max_len-1),   #  고유단어의 갯수, embedding vector length, 입력 문장의 단어수
    SimpleRNN(32, activation='tanh'),   # activation='tanh' 는 생략 가능
    Dense(vocab_size, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5, 4)              48        
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                1184      
                                                                 
 dense (Dense)               (None, 12)                396       
                                                                 
Total params: 1628 (6.36 KB)
Trainable params: 1628 (6.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
model.compile('adam', 'categorical_crossentropy', ['accuracy'])
hist = model.fit(X, Y, epochs=200, verbose=0)

In [17]:
hist.history['accuracy'][-1]        # 정확도 90.1%

0.9090909361839294

- 모델 검증

In [18]:
from my_util import sentence_generation

sentence_generation(model, t, max_len, '경마장에', 3)   # 모델, 토크나이저, maxlen, 입력단어, 반복횟수

'경마장에 말이 고와야 오는'

In [19]:
sentence_generation(model, t, max_len, '가는', 5)

'가는 말이 고와야 오는 말이 곱다'

In [20]:
sentence_generation(model, t, max_len, '그의', 2)

'그의 말이 법이다'

#### 모델 변화
- Embedding vector length: [2, 4, 6]
- RNN 유닛 갯수: [24, 32, 48]

In [22]:
for n_embed in [2,4,6]:
    for n_unit in [24,32,48]:
        model = Sequential([
            Embedding(vocab_size, n_embed, input_length=max_len-1),    
            SimpleRNN(n_unit),
            Dense(vocab_size, activation='softmax')
        ])
        model.compile('adam', 'categorical_crossentropy', ['accuracy'])
        hist = model.fit(X, Y, epochs=200, verbose=0)
        print('==================================================')
        print(f'Embedding vector: {n_embed}, RNN unit: {n_unit}')
        print(f"정확도: {hist.history['accuracy'][-1]:.4f}")
        print(sentence_generation(model, t, max_len, '경마장에', 4))
        print(sentence_generation(model, t, max_len, '그의', 2))
        print(sentence_generation(model, t, max_len, '가는', 5))
        print()

Embedding vector: 2, RNN unit: 24
정확도: 0.9091
경마장에 말이 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector: 2, RNN unit: 32
정확도: 0.8182
경마장에 말이 말이 뛰고 있다
그의 말이 법이다
가는 말이 말이 오는 말이 곱다

Embedding vector: 2, RNN unit: 48
정확도: 0.9091
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 법이다 오는 말이 곱다

Embedding vector: 4, RNN unit: 24
정확도: 0.9091
경마장에 말이 법이다 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector: 4, RNN unit: 32
정확도: 1.0000
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector: 4, RNN unit: 48
정확도: 0.9091
경마장에 말이 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector: 6, RNN unit: 24
정확도: 1.0000
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector: 6, RNN unit: 32
정확도: 1.0000
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector: 6, RNN unit: 48
정확도: 1.0000
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

